In [55]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.metrics import AUC
from keras.callbacks import EarlyStopping
import pandas as pd
from pathlib import Path
import os
import scipy
from scipy import stats
from sklearn import preprocessing
import numpy as np
os.getcwd()

'/Users/rsfletch/github/bts/notebooks'

In [56]:
def prep_data(df):
    # 'HPG', 'HPAB_p', 'spot', 'home', 'factor', 'BAT_HAND', 'PIT_HAND', 'avg_win', 'year'
    cols = [
        'spot', 'home', 'b_HPG', 'p_HPAB', 'park_factor', 'year',
        'BAT_HAND', 'PIT_HAND', 'b_avg_win', 'p_team_HPAB',
        'p_avg_game_score', 'p_team_avg_game_score'
    ]
    num_cols = len(cols)
    num_feat = num_cols - 1
    
    
    df[['home']] = df[['home']]*1
    X = df[cols]*1
    Y = df[['Win']]*1
    X_train = X[(df.year >= 2000) & (df.year < 2010)].to_numpy().astype('float') # (df.year >= 2000) & 
    Y_train = Y[(df.year >= 2000) & (df.year < 2010)].to_numpy().astype('float') # (df.year >= 2000) & 
    X_test = X[df.year >=  2010].to_numpy().astype('float')
    Y_test = Y[df.year >=  2010].to_numpy().astype('float')

    mean = np.mean(X_train, axis=0)
    X_train -= mean
    X_test -= mean
    std = np.std(X_train, axis=0)
    print(std)
    X_train /= std
    X_test /= std
    
    return X_train, Y_train, X_test, Y_test

In [57]:
processed = '../data/processed'
main_data = pd.read_pickle(Path(processed) / 'main_data.pkl')

main_data = main_data.dropna()

X_train, Y_train, X_test, Y_test = prep_data(main_data)

[2.50616573 0.49999972 0.25387502 0.03705335 0.07224307 2.8330341
 0.49354651 0.4464489  0.11226924 0.03653893 5.29353691 5.31655443]


In [58]:
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(18, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[AUC()])

In [59]:
callback = EarlyStopping(monitor='loss', patience=2)
model.fit(X_train, Y_train, epochs=30, batch_size=64, callbacks=[callback])
print('done')

Epoch 1/30
295714/295714 [==============================] - 15s 49us/step - loss: 0.6316 - auc_9: 0.5772
Epoch 2/30
295714/295714 [==============================] - 14s 47us/step - loss: 0.6275 - auc_9: 0.5870
Epoch 3/30
295714/295714 [==============================] - 15s 50us/step - loss: 0.6270 - auc_9: 0.5886
Epoch 4/30
295714/295714 [==============================] - 14s 48us/step - loss: 0.6268 - auc_9: 0.5892
Epoch 5/30
295714/295714 [==============================] - 14s 47us/step - loss: 0.6266 - auc_9: 0.5899
Epoch 6/30
295714/295714 [==============================] - 13s 45us/step - loss: 0.6265 - auc_9: 0.5901
Epoch 7/30
295714/295714 [==============================] - 14s 48us/step - loss: 0.6265 - auc_9: 0.5902
Epoch 8/30
295714/295714 [==============================] - 14s 48us/step - loss: 0.6264 - auc_9: 0.5905
Epoch 9/30
295714/295714 [==============================] - 14s 46us/step - loss: 0.6264 - auc_9: 0.5903
Epoch 10/30
295714/295714 [============================

In [60]:
model.summary()
_, accuracy = model.evaluate(X_test, Y_test)
print('Accuracy: %.2f' % (accuracy*100))

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 12)                156       
_________________________________________________________________
dense_26 (Dense)             (None, 18)                234       
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 19        
Total params: 409
Trainable params: 409
Non-trainable params: 0
_________________________________________________________________
312791/312791 [==============================] - 15s 50us/step
Accuracy: 58.57


In [61]:
predictions = model.predict(X_test)
test_data = main_data[main_data.year >=  2010]
test_data['EstProb'] = predictions
test_data = test_data.set_index(['GAME_ID', 'BAT_ID'])

/Users/rsfletch/opt/anaconda3/envs/bts/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
selection = test_data.groupby('Date')['EstProb'].nlargest(2).to_frame()

selection = selection.sort_values(
    by=['Date', 'EstProb', 'GAME_ID'], ascending=[True, False, True])

selection['pick_order'] = selection.groupby(['Date']).cumcount()+1
selection.to_pickle('../data/processed/selection_data.pkl')

In [63]:
interim = '../data/interim'
processed = '../data/processed'

hits = pd.read_pickle(Path(interim) / 'hits.pkl')

# selection_data = selections.merge(hits, on =['GAME_ID', 'BAT_ID'])
selection_data = selection.merge(main_data, on =['GAME_ID', 'BAT_ID'])
selection_data = selection_data.set_index(['Date', 'pick_order'])

selection_data.to_pickle(Path(processed) / 'selection_data.pkl')